In [1]:
import tensorflow as tf
import os
import _pickle as cPickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"

from myUtils.utils import CifarData

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(10)
print(batch_data)
print(batch_labels)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)
[[ 0.76470588  0.74901961  0.74901961 ... -0.49019608 -0.56078431
  -0.62352941]
 [-0.41176471 -0.41960784 -0.41960784 ...  0.39607843  0.40392157
   0.42745098]
 [ 0.75686275  0.63137255  0.61568627 ... -0.11372549 -0.10588235
  -0.1372549 ]
 ...
 [ 0.6627451   0.67058824  0.69411765 ... -0.6        -0.4745098
  -0.5372549 ]
 [-0.64705882 -0.70196078 -0.74117647 ... -0.60784314 -0.61568627
  -0.61568627]
 [-0.63137255 -0.78039216 -0.81960784 ...  0.08235294  0.10588235
   0.05882353]]
[3 0 7 1 2 6 9 3 8 4]


In [2]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
x_image = tf.reshape(x, [-1, 3, 32, 32])
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

conv1 = tf.layers.conv2d(x_image, 32, (3, 3), padding='same', activation=tf.nn.relu, name='conv1')
pooling1 = tf.layers.max_pooling2d(conv1, (2, 2), (2, 2), name='pool1')
conv2 = tf.layers.conv2d(pooling1, 32, (3, 3), padding='same', activation=tf.nn.relu, name='conv2')
pooling2 = tf.layers.max_pooling2d(conv2, (2, 2), (2, 2), name='pool2')
conv3 = tf.layers.conv2d(pooling2, 32, (3, 3), padding='same', activation=tf.nn.relu, name='conv3')
pooling3 = tf.layers.max_pooling2d(conv3, (2, 2), (2, 2), name='pool3')

flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)


loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

predict = tf.argmax(y_, 1) 
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [4]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 1000
test_steps = 100
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, accu_val, _ = sess.run([loss, accuracy, train_op], feed_dict={x: batch_data, y: batch_labels})
        if (i + 1) % 100 == 0 :
            print("[Train] Step: %d, loss: %4.5f, accuracy: %4.5f" % (i + 1, loss_val, accu_val))
        if (i + 1) % 500 == 0 :
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps) :
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy], feed_dict={x: test_batch_data, y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print("[Test] Step: %d, accuracy: %4.5f" % (i + 1, test_acc))

[Train] Step: 100, loss: 1.84091, accuracy: 0.30000
[Train] Step: 200, loss: 1.77469, accuracy: 0.30000
[Train] Step: 300, loss: 1.70990, accuracy: 0.45000
[Train] Step: 400, loss: 2.20998, accuracy: 0.20000
[Train] Step: 500, loss: 1.35487, accuracy: 0.40000
(10000, 3072)
(10000,)
[Test] Step: 500, accuracy: 0.45850
[Train] Step: 600, loss: 1.68966, accuracy: 0.25000
[Train] Step: 700, loss: 1.42696, accuracy: 0.25000
[Train] Step: 800, loss: 1.44431, accuracy: 0.50000
[Train] Step: 900, loss: 1.68999, accuracy: 0.35000
[Train] Step: 1000, loss: 0.83679, accuracy: 0.70000
(10000, 3072)
(10000,)
[Test] Step: 1000, accuracy: 0.53000
